In [1]:
import pandas as pd
import numpy as np
#import featuretools as ft
import calendar
#import xmltodict

In [2]:
#
# справочник выходных и праздничных дней с 1999 до 2025 гг.

work_days = pd.read_csv('work.csv', encoding='1251', index_col='Год/Месяц')

# словарь национальный праздников РФ

dict_hol = {101: 'НГ', 102: 'НГ', 103: 'НГ', 104: 'НГ', 105: 'НГ', 106: 'НГ',
     107: 'Рожд', 223: '23фев', 308: '8мар', 501: '1мая', 509: '9мая', 612: '12ию', 114: '4ноя'}

In [3]:
#
# признаки из дейттайм

def datefeatures(df, i = 'index', date = 0, time = 1, dist = 1):
    
    dataframe = pd.DataFrame(df)
    
    # признаки из дат
    
    if date == 0:
    
        # номер года

        dataframe[i + '_year'] = df.dt.year

        # номер месяца

        dataframe[i + '_mnth'] = df.dt.month

        # номер недели

        dataframe[i + '_week'] = df.dt.week 

        # 1 декада?

        dataframe[i + '_is_dcd1'] = df.map(lambda x: 1 if x.date().day < 11 else 0)

        # 2 декада?

        dataframe[i + '_is_dcd2'] = df.map(lambda x: 1 if 10 < x.date().day < 21 else 0)

        # номер дня недели

        dataframe[i + '_dow'] = df.apply(lambda x: x.date().weekday())

        # номер дня месяца (ВНИМАНИЕ! большая нагрузка на память при переходе к дамми)

        dataframe[i + '_day'] = df.dt.day

        # сб или вс?

        dataframe[i + '_is_eow'] = df.apply(lambda x: 1 if x.date().weekday() in (5, 6) else 0)
        
        # выходной (1999-2025 гг.)?
        
        try:
            dataframe[i + '_is_wknd'] = dataframe[i].apply(lambda x: int(str(x.day) in work_days[str(x.month)][x.year].split(',')))
        except:
            pass
        
        # национальные праздники
        
        dataframe[i + '_ruhol'] = dataframe[i].map(lambda x: dict_hol[x.month*100+x.day] if dict_hol.get(x.month*100+x.day) else 'нет')


        # при необходимости сохранить дистанции

        if dist == 1: 

            # день месяца в sin, cos

            dataframe[i + '_day_cos'] = df.apply(lambda x: make_harmonic_features(x.day, \
                                    calendar.monthrange(x.year, x.month)[1])[0])
            dataframe[i + '_day_sin'] = df.apply(lambda x: make_harmonic_features(x.day, \
                                    calendar.monthrange(x.year, x.month)[1])[1])

            # номер месяца в sin, cos

            dataframe[i + '_mnth_cos'] = make_harmonic_features(df.dt.month, 12)[0]
            dataframe[i + '_mnth_sin'] = make_harmonic_features(df.dt.month, 12)[1]

    
    
    # признаки из времени

    if time == 0:

        # час

        dataframe[i + '_hr'] = df.dt.hour

        # минута 

        dataframe[i + '_mnt'] = df.dt.minute

        # секунда 

        dataframe[i + '_sec'] = df.dt.second
        

        # при необходимости сохранить дистанции
        
        if dist == 0:
            
            
            # час в sin, cos
            
            dataframe[i + '_hr_cos'] = df.apply(lambda x: make_harmonic_features(x.hour, 24)[0])
            dataframe[i + '_hr_sin'] = df.apply(lambda x: make_harmonic_features(x.hour, 24)[1])
            
            # минута в sin, cos
            
            dataframe[i + '_mnt_cos'] = df.apply(lambda x: make_harmonic_features(x.minute, 60)[0])
            dataframe[i + '_mnt_sin'] = df.apply(lambda x: make_harmonic_features(x.minute, 60)[1])

            
            

    return dataframe[dataframe.columns[1:]]
  
        
# эзотерический подход

def make_harmonic_features(value, period):
    
    value *= 2 * np.pi / period
    return np.cos(value), np.sin(value)



# на вход подается pandas.DataFrame с именем train

In [44]:
train = pd.read_csv('C:/Users/Noname/sdsj/data/check_4_c/train.csv', parse_dates=True)

#
# выделить столбцы с датами в отдельный датасет

dfd = train.filter(regex='date*').apply(pd.to_datetime)
#dfd.info()
dfd_columns = dfd.columns

In [45]:
#
# дополнительно разности дат, если несколько полей с датами

if dfd.shape[1] > 0:
    
    for i in range(len(dfd_columns)):
        for j in dfd_columns[i + 1:]:
            
            dfd['diffdate_' + str(i) + '_' + str(j)] = (dfd[dfd_columns[i]] - dfd[j]).dt.seconds//60 + (dfd[dfd_columns[i]] - dfd[j]).dt.days*24*60

In [19]:
#
# шаблон с индексами исходного

dfd_feat = pd.DataFrame(index = dfd.index)

# утилита работает постолбчато

for feat in dfd_columns:
    
    df_temp = (datefeatures(dfd[feat], i = feat, \
                            date = (dfd[feat].iloc[0].date() == dfd[feat].iloc[-1].date() == pd.to_datetime('00:00:00').date()), \
                            time = (dfd[feat].iloc[0].time() == dfd[feat].iloc[-1].time() == pd.to_datetime('2000-01-01').time()), \
                            dist = 1))
    dfd_feat = dfd_feat.join(df_temp, how='outer')
    df_temp.drop(df_temp.index, inplace = True)

In [46]:
dfd.tail()

,datetime_0,datetime_1,datetime_2,diffdate_0_datetime_1,diffdate_0_datetime_2,diffdate_1_datetime_2
114125,2017-05-26,2017-05-02,2018-04-05,34560,-452160,-486720
114126,2017-06-17,2017-05-30,2017-06-25,25920,-11520,-37440
114127,2017-05-06,2017-04-17,2017-05-11,27360,-7200,-34560
114128,2017-06-16,2017-05-30,2018-04-05,24480,-421920,-446400
114129,2017-04-12,2017-03-15,2018-04-05,40320,-515520,-555840


In [40]:
a = pd.to_datetime('02.12.2013')
b = pd.to_datetime('02.08.2013')
print(a)
print(b)
print((a-b))
print((a-b).seconds//60)
print((a-b).days)
print((a-b).seconds//60 + (a-b).days*24*60)

2013-02-12 00:00:00
2013-02-08 00:00:00
4 days 00:00:00
0
4
5760
